In [1]:
from __future__ import print_function
import sys, os
import h2o
from h2o.estimators.deepwater import H2ODeepWaterEstimator

import importlib
h2o.init(url="http://localhost:54327")

Checking whether there is an H2O instance running at http://localhost:54327..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_101"; Java(TM) SE Runtime Environment (build 1.8.0_101-b13); Java HotSpot(TM) 64-Bit Server VM (build 25.101-b13, mixed mode)
  Starting server from /Users/arno/h2o-3/build/h2o.jar
  Ice root: /var/folders/fm/9q0qk_xs0wd07zr2v6d5mfv40000gn/T/tmp7hkqOj
  JVM stdout: /var/folders/fm/9q0qk_xs0wd07zr2v6d5mfv40000gn/T/tmp7hkqOj/h2o_arno_started_from_python.out
  JVM stderr: /var/folders/fm/9q0qk_xs0wd07zr2v6d5mfv40000gn/T/tmp7hkqOj/h2o_arno_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:92: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:98: DeprecationWarning: DisplayFormatter._formatters_default is deprecated: use @default decorator instead.
  def _formatters_default(self):
/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:677: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):
/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):
/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:672: DeprecationWarning: PlainTextFormat

H2O cluster uptime:,01 secs
H2O cluster version:,3.11.0.99999
H2O cluster version age:,1 minute
H2O cluster name:,H2O_from_python_arno_onzhms
H2O cluster total nodes:,1
H2O cluster free memory:,3.556 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://127.0.0.1:54321
H2O connection proxy:,None


# LeNET

Here we define the famouse LENET neural network

In [2]:
def lenet(num_classes):
    import mxnet as mx
    data = mx.symbol.Variable('data')
    # first conv
    conv1 = mx.symbol.Convolution(data=data, kernel=(5,5), num_filter=20)
    tanh1 = mx.symbol.Activation(data=conv1, act_type="tanh")
    pool1 = mx.symbol.Pooling(data=tanh1, pool_type="max", kernel=(2,2), stride=(2,2))
    # second conv
    conv2 = mx.symbol.Convolution(data=pool1, kernel=(5,5), num_filter=50)
    tanh2 = mx.symbol.Activation(data=conv2, act_type="tanh")
    pool2 = mx.symbol.Pooling(data=tanh2, pool_type="max", kernel=(2,2), stride=(2,2))
    # first fullc
    flatten = mx.symbol.Flatten(data=pool2)
    fc1 = mx.symbol.FullyConnected(data=flatten, num_hidden=500)
    tanh3 = mx.symbol.Activation(data=fc1, act_type="tanh")
    # second fullc
    fc2 = mx.symbol.FullyConnected(data=tanh3, num_hidden=num_classes)
    # loss
    lenet = mx.symbol.SoftmaxOutput(data=fc2, name='softmax')
    return lenet

## DeepWater for MXNET

We can use the import functions provided by h2o to import the list of 

In [3]:
train = h2o.import_file("../../bigdata/laptop/mnist/train.csv.gz")
test = h2o.import_file("../../bigdata/laptop/mnist/test.csv.gz")

predictors = list(range(0,784))
resp = 784
train[resp] = train[resp].asfactor()
test[resp] = test[resp].asfactor()
nclasses = train[resp].nlevels()[0]

Parse progress: |█████████████████████████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████████████████████████| 100%


Let's create the lenet model architecture from scratch using the MXNet Python API

In [4]:
model = lenet(nclasses)

To import the model inside the DeepWater training engine we need to save the model to a file:

In [5]:
model_path = "/tmp/symbol_lenet-py.json"
model.save(model_path)

The model is just the structure of the network expressed as a json dict

In [6]:
!head "/tmp/symbol_lenet-py.json"

{
  "nodes": [
    {
      "op": "null", 
      "param": {}, 
      "name": "data", 
      "inputs": [], 
      "backward_source_id": -1
    }, 
    {


## Importing the LeNET model architecture for training in H2O

We have defined the model and saved the structure to a file. We are ready to start the training procedure.

In [7]:
model = H2ODeepWaterEstimator(epochs=100, rate=1e-3, 
                              mini_batch_size=64,
                              network='user', 
                              network_definition_file=model_path,
                              image_shape=[28,28], channels=1)

In [ ]:
model.train(x=predictors,y=resp, training_frame=train, validation_frame=test)

deepwater Model Build progress: |█████████

In [ ]:
model.show()